# Setting Up the Transfer Functions

In [27]:
%load_ext autoreload
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Notebook Initialization

In [28]:
%autoreload
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from darkhistory.spec.spectrum import Spectrum
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
import darkhistory.physics as phys
import darkhistory.utilities as utils

np.set_printoptions(threshold=np.nan)

## Import Raw Data

In [29]:
xes_str = [
    '4.540E-05', '2.746E-04', '1.659E-03', '9.952E-03', '5.732E-02',
    '2.689E-01', '6.900E-01', '9.309E-01', '9.879E-01', '9.980E-01',
    '9.997E-01'
    
]
file_names = [
    '/Users/hongwan/Dropbox (MIT)/Photon Deposition/transferfunction_withloweng_cmbloss_60eV_xe_'+str+'_nointerp.p'
    for str in xes_str
    
]
raw_tf_tuple = [pickle.load(open(file_name, "rb")) for file_name in file_names]

In [30]:
print(raw_tf_tuple[0][0].shape)
print(raw_tf_tuple[0][1].shape)
print(raw_tf_tuple[1][0].shape)
print(raw_tf_tuple[1][1].shape)

(500, 51, 500, 3)
(500, 51)
(500, 51, 500, 3)
(500, 51)


In [31]:
raw_tf_list = [tup[0] for tup in raw_tf_tuple]
CMB_engloss_list = [tup[1] for tup in raw_tf_tuple]

## Raw Data Manipulation

The raw data is a large array with no abscissa values included. We need to create the abscissae related to this large array (the abscissa of photon energies and electron energies). 

First, we swap some of the axes. The initial array has dimensions corresponding to (injected photon energy, redshift, outgoing energy, type), where type is high-energy photons, low-energy photons and low-energy electrons respectively. 

In [34]:
for i in np.arange(len(raw_tf_list)):
    raw_tf_list[i] = np.swapaxes(raw_tf_list[i], 0, 1)
    raw_tf_list[i] = np.flip(raw_tf_list[i], axis=0)

In [35]:
for i in np.arange(len(CMB_engloss_list)):
    CMB_engloss_list[i] = np.transpose(CMB_engloss_list[i])

At this point, the order is now (redshift, input photon energy, output energy, type), and the order is in decreasing redshift.

The abscissa for the transfer functions are as follows. For the injected photon energy and redshifts,

In [36]:
# Input energy abscissa. 
in_eng_step = 500
# in_eng_low = 3e3 + 100.
in_eng_low = 60
in_eng_upp = 5e3 * np.exp(39 * np.log(1e13/5e3) / 40)

in_eng_arr = (
    in_eng_low * np.exp((np.arange(in_eng_step)) * 
              np.log(in_eng_upp/in_eng_low) / in_eng_step)
)

# Redshift abscissa, decreasing order. 
rs_step = 50
rs_upp  = 31
rs_low  = 4

log_rs = (
    np.log(rs_low) + (np.arange(rs_step) + 1)
    *(np.log(rs_upp) - np.log(rs_low))/rs_step
)

log_rs_arr = np.flipud(log_rs)

# xe abscissa

xe = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

The output abscissa depends on the injection energy, and are different for photons and electrons. We write these as functions of the injection energy for convenience.

In [37]:
def get_out_photeng(in_eng):
    
    log_bin_width = np.log((phys.me + in_eng)/1e-4)/500
    bin_boundary  = 1e-4 * np.exp(np.arange(501) * log_bin_width)
    bin_boundary_low = bin_boundary[0:500]
    bin_boundary_upp = bin_boundary[1:501]

    return np.sqrt(bin_boundary_low * bin_boundary_upp)

def get_out_eleceng(in_eng):
    
    log_bin_width = np.log(in_eng)/500
    bin_boundary  = phys.me + np.exp(np.arange(501) * log_bin_width)
    bin_boundary_low = bin_boundary[:500]
    bin_boundary_upp = bin_boundary[1:]
    return np.sqrt(
        (bin_boundary_low - phys.me)*(bin_boundary_upp - phys.me)
    )    

The transfer function is calculated by injecting 2 photons at the input energy abscissa. However, the first step is to assign $x$ photons to the top bin of the *output* photon abscissa, such that $x$ times the energy of the top bin is 2 times the injection energy. Because of the misalignment between the two abscissae, as well as the fact that we want to transfer function for a single electron, we have to normalize the results first.

First, we construct two arrays: a list of output abscissae (for both photons and electrons) given the injection abscissa, and a list of the energy of the *output* photon energy bin where the injected photons are assigned to. Then we compute the normalization factor. 

In [38]:
# dimensions input x output
# output energy abscissae
out_photeng_arr = np.array([get_out_photeng(eng) for eng in in_eng_arr])
# dimensions input x output
out_eleceng_arr = np.array([get_out_eleceng(eng) for eng in in_eng_arr])

# dimensions input
# original injection energies
top_photeng_bin_arr = np.array(
    [
        photeng[photeng < eng][-1] 
        for eng,photeng in zip(in_eng_arr, out_photeng_arr)
    ]
)

# dimensions input
norm_fac_arr = in_eng_arr/top_photeng_bin_arr*2

Dividing by the normalization factor gives the spectra for the injection of 1 photon, with energy given by top_photeng_bin_arr. 

Now we can construct the raw `Spectrum` lists that we will finally put into a `TransFuncAtEnergy` object for the high energy photons, low energy photons and low energy electrons. These are spectra with input energy given by dN/dE = 1 for each bin of `in_eng_arr`. 

In [39]:
photspec_list = [
    [
        [
            Spectrum(
                out_photeng, raw_tf[i,j,:,0]/norm_fac, 
                rs = np.exp(log_rs), in_eng = in_eng
            ) 
            for (i, log_rs) in enumerate(log_rs_arr)
        ] for (j, (in_eng, out_photeng, norm_fac)) in enumerate(
                zip(top_photeng_bin_arr, out_photeng_arr, norm_fac_arr)
        )
    ] for raw_tf in tqdm(raw_tf_list)
]

lowengphotspec_list = [
    [
        [
            Spectrum(
                out_photeng, raw_tf[i,j,:,1]/norm_fac, 
                rs = np.exp(log_rs), in_eng = in_eng
            ) 
            for (i, log_rs) in enumerate(log_rs_arr)
        ] for (j, (in_eng, out_photeng, norm_fac)) in enumerate(
                zip(in_eng_arr, out_photeng_arr, norm_fac_arr)
        )
    ] for raw_tf in tqdm(raw_tf_list)
]

lowengelecspec_list = [
    [
        [
            Spectrum(
                out_eleceng, raw_tf[i,j,:,2]/norm_fac, 
                rs = np.exp(log_rs), in_eng = in_eng
            ) 
            for (i, log_rs) in enumerate(log_rs_arr)
        ] for (j, (in_eng, out_eleceng, norm_fac)) in enumerate(
                zip(in_eng_arr, out_eleceng_arr, norm_fac_arr)
        )
    ] for raw_tf in tqdm(raw_tf_list)
]

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

KeyboardInterrupt: 

Now we rebin all of the `Spectrum` objects. The final abscissa that we would like to use is `get_out_photeng(in_eng_arr[-1])`, i.e. the photon abscissa corresponding to the largest injected energy, as well as `get_out_eleceng(in_eng_arr[-1])` for the electrons. We also change the `Spectrum` object to type `N`.

In [22]:
fin_photeng = get_out_photeng(in_eng_arr[-1])
fin_eleceng = get_out_eleceng(in_eng_arr[-1])

for phot_specs_xe in tqdm(photspec_list):
    for phot_specs in tqdm(phot_specs_xe):
        for phot_spec in phot_specs:
#             phot_spec.rebin(top_photeng_bin_arr)
            phot_spec.rebin(fin_photeng)
            phot_spec.switch_spec_type()

for phot_specs_xe in tqdm(lowengphotspec_list):
    for phot_specs in tqdm(phot_specs_xe):
        for phot_spec in phot_specs:
            phot_spec.rebin(fin_photeng)
            phot_spec.switch_spec_type()

for elec_specs_xe in tqdm(lowengelecspec_list):
    for elec_specs in tqdm(elec_specs_xe):
        for elec_spec in elec_specs:
            elec_spec.rebin(fin_eleceng)
            elec_spec.switch_spec_type()
    

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

Finally, we can construct the `TransferFuncList` from this.

In [23]:
tfunclist_photspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.002)
            for spec_arr in photspec_xe
        ]
    ) for photspec_xe in tqdm(photspec_list)
]

tfunclist_lowengphotspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.002)
            for spec_arr in lowengphotspec_xe
        ]
    ) for lowengphotspec_xe in tqdm(lowengphotspec_list)
]
    

tfunclist_lowengelecspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.002)
            for spec_arr in lowengelecspec_xe
        ]
    ) for lowengelecspec_xe in tqdm(lowengelecspec_list)
]

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

We will now transpose all of the `TransFuncList` objects so that they are now all of type `'rs'`. This will then 

In [24]:
for tflist_hep, tflist_lep, tflist_lee in zip(
    tfunclist_photspec_list, tfunclist_lowengphotspec_list, tfunclist_lowengelecspec_list):
    tflist_hep.transpose()
    tflist_lep.transpose()
    tflist_lee.transpose()
    for tf_hep, tf_lep, tf_lee in zip(tflist_hep, tflist_lep, tflist_lee):
        tf_hep.grid_vals[np.where(tf_hep.grid_vals < 0)] = 1e-100
        tf_lep.grid_vals[np.where(tf_lep.grid_vals < 0)] = 1e-100
        tf_lee.grid_vals[np.where(tf_lee.grid_vals < 0)] = 1e-100

In [25]:
pickle.dump(tfunclist_photspec_list, 
           open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_list.raw", "wb")
           )

pickle.dump(tfunclist_lowengphotspec_list, 
           open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_list.raw", "wb")
           )

pickle.dump(tfunclist_lowengelecspec_list, 
           open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_list.raw", "wb")
           )

At the end, we have the normalized transfer functions, which will takes in a `Spectrum` of number of particles in each bin, and outputs the `Spectrum`, also number of particles.

In [26]:
pickle.dump(
    CMB_engloss_list, 
    open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/CMB_engloss_list.raw", "wb")       
           )

In [ ]:
tfunclist_photspec_list[0].eng